# Le problème de la couverture exacte

## Objet du problème

$F$ étant un ensemble de parties non vides d'un ensemble fini $E$,
il s'agit de trouver une partie $C$ de 
$F$ qui constitue une partition de $E$ (les éléments de $C$ sont 2 à 2 disjoints et leur réunion est $E$).
On dit que $C$ est une *couverture exacte*. 

Par exemple, avec $E=\{0,1,2,3,4,5,6\}$, une couverture exacte de
$F=\{\{2,4,5\},\{0,3,6\},\{1,2,5\},\{0,3\},\{1,6\},\{3,4,6\}\}$ est
$C=\{\{2,4,5\},\{0,3\},\{1,6\}\}$.

## L'algorithme X

couverture-exacte $\!(E,F)$ :  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** $E=\emptyset$ **alors**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**renvoyer** $\emptyset$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**sinon**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**choisir** $x\in E$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**pour tout** $P$ tel que $x\in P\in F$  **faire**    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$E'\leftarrow E\setminus P$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$F'\leftarrow\{Q\in F\,,\,P\cap Q=\emptyset\}$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** couverture-exacte $\!(E',F')$ renvoie $C$ **alors**   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**renvoyer** $C\cup\{P\}$

Cet algorithme termine car le cardinal de $E'$ dans
les appels récursifs est strictement inférieur à celui de $E$.  
La structure même de l'algorithme fournit une preuve de sa validité.

## Implémentation

### Méthode élémentaire
On peut implémenter l'algorithme en Python, voir par exemplee [ici](https://www.cs.mcgill.ca/~aassaf9/python/algorithm_x.html). 

### Méthode de Knuth
[Donald Knuth](https://www-cs-faculty.stanford.edu/~knuth/) a mis au point une méthode efficace : les [liens dansants](https://arxiv.org/pdf/cs/0011047.pdf). Mais cette méthode, qui est basée sur une utilisation astucieuse de pointeurs,  perd son efficacité si on l'implémente en pur Python et il faut utiliser le langage C. A ma connaissance, il n'existe pas de module Python implémentant les liens dansants en C. Par contre, un tel module existe en [Sage](https://www.sagemath.org/). Le code suivant est environ 10 fois plus rapide que toute implémentation pur Python.

In [4]:
class AlgorithmeX:

    def __init__(self, F, obl = [], fac = []):

        """F est un dictionnaire d'objets hashables :
        {<choix> : [<contrainte>,...],...}
        obl est la liste des choix obligatoires.
        fac est la liste des contraintes facultatives.
        """

        self.dictFac = {'_facultatif' + str(i) : [k] for i,k in enumerate(fac)}
        self.F = {**F, **self.dictFac}
        E = set() 
        for l in self.F: E |= set(self.F[l])  
        self.E = list(E)    
        self.cardF = len(self.F)
        self.obl = obl

    def solve(self):
        """Renvoie un iterateur des solutions 
        """
        from sage.combinat.matrices.dancing_links import dlx_solver
        dictE = {k : int(i) for i, k in enumerate(self.E)}
        keysF = list(self.F.keys())
        dictF = {k : int(i) for i, k in enumerate(self.F)}
        entiersF = [list(map((lambda o: dictE[o]),self.F[keysF[i]])) for i in range(self.cardF)]
        def solutions():
            for s in dlx_solver(entiersF).restrict(map((lambda o: dictF[o]),self.obl)).solutions_iterator():
                yield list(set(map((lambda i: keysF[i]),s)) - set(self.dictFac.keys()))
        return solutions()

    def printSolution(self,sol):
        for l in sol: print(self.F[l])            

### Test 1

In [5]:
F = {'1': [1,2],'2': [3,4,5],'3': [0,1],'4': [2,3,4,5],'5': [0],'6': [1,2,3,4,5,6]}
aX = AlgorithmeX(F,obl=['5'],fac=[6])
#for sol in aX.solve(): print(sol)       

### Test 2

In [6]:
F = {'A' : ['1', 4, '7'],
     'B' : ['1', 4],
     'C' : [4, '5', '7'],
     'D' : [ '5','3', '6'],
     'E' : [ (3,4), '3', '6', '7'],
     'F' : [ (3,4), '7']}

aX = AlgorithmeX(F)    
#for sol in aX.solve(): aX.printSolution(sol)    